In [1]:
import json
import os
import pinecone
import re
import requests
from sentence_transformers import SentenceTransformer
from statistics import mean
import swifter
import pandas as pd

In [2]:
from flask import Flask

In [3]:
app = Flask(__name__)

PINECONE_INDEX_NAME = "plagiarism-checker"
DATA_FILE = "articles.csv"
NROWS = 20000

In [4]:
import pinecone
pinecone.init(api_key="68ffb5f6-a308-443d-a900-fe671332783f", environment="us-west1-gcp")

In [5]:
pinecone.list_indexes()

['plagiarism-checker']

In [6]:
def delete_existing_pinecone_index():
    if plagiarism-checker in pinecone.list_indexes():
        pinecone.delete_index(plagiarism-checker)

In [7]:
def create_pinecone_index():
    pinecone.create_index(name=Plagiarism-Detection, metric="cosine", shards=1)
    pinecone_index = pinecone.Index(name=Plagiarism-Detection)

    return pinecone_index

In [8]:
def create_model():
    model = SentenceTransformer('average_word_embeddings_komninos')

    return model

In [9]:
def prepare_data(data):
    # rename id column and remove unnecessary columns
    data.rename(columns={"Unnamed: 0": "article_id"}, inplace = True)
    data.drop(columns=['date'], inplace = True)

    # combine the article title and content into a single field
    data['content'] = data['content'].fillna('')
    data['content'] = data.content.swifter.apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)))
    data['title_and_content'] = data['title'] + ' ' + data['content']

    # create a vector embedding based on title and article content
    encoded_articles = model.encode(data['title_and_content'], show_progress_bar=True)
    data['article_vector'] = pd.Series(encoded_articles.tolist())

    return data

In [10]:
def upload_items(data):
    items_to_upload = [(row.id, row.article_vector) for i, row in data.iterrows()]
    pinecone_index.upsert(items=items_to_upload)

In [11]:
def process_file(filename):
    data = pd.read_csv("C:\\Users\\rafyq\\OneDrive\\Desktop\\archive (2)\\articles1.csv", nrows=NROWS)
    data = prepare_data(data)
    upload_items(data)
    pinecone_index.info()

    return data

In [12]:
def map_titles(data):
    return dict(zip(uploaded_data.id, uploaded_data.title))

In [13]:
def map_publications(data):
    return dict(zip(uploaded_data.id, uploaded_data.publication))

In [15]:
def query_pinecone(originalContent):
    query_content = str(originalContent)
    query_vectors = [model.encode(query_content)]

    query_results = pinecone_index.query(queries=query_vectors, top_k=10)
    res = query_results[0]

    results_list = []

    for idx, _id in enumerate(res.ids):
        results_list.append({
            "id": _id,
            "title": titles_mapped[int(_id)],
            "publication": publications_mapped[int(_id)],
            "score": res.scores[idx],
        })

    return json.dumps(results_list)

In [ ]:
pinecone.init()
model = create_model()
uploaded_data = process_file(filename=articles1)
titles_mapped = map_titles(uploaded_data)
publications_mapped = map_publications(uploaded_data)

In [17]:
@app.route("/")
def index():
    return render_template("index.html")

In [18]:
@app.route("/api/search", methods=["POST", "GET"])
def search():
    if request.method == "POST":
        return query_pinecone(request.form.get("originalContent", ""))
    if request.method == "GET":
        return query_pinecone(request.args.get("originalContent", ""))
        return "Only GET and POST methods are allowed for this endpoint"